# Building a crew

The following code suppresses warning messages to ensure the output is clean and focused on the essential results.

In [1]:
import warnings
import os
warnings.filterwarnings('ignore')
os.environ['PIP_ROOT_USER_ACTION'] = 'ignore'

**Setup note:** Keep your API keys out of the notebook. Set `OPENAI_API_KEY` and `SERPER_API_KEY` in your shell or a `.env` (loaded before runtime). If unset, the next cell will securely prompt you once per session.


Imports essential modules from CrewAI. Also, sets up environment variables to securely store API keys required for accessing external services.

In [2]:
from crewai import Agent, Task, Crew
from crewai_tools import SerperDevTool
import os, getpass
from dotenv import load_dotenv

# Load keys from the project .env; fallback to prompting once if missing
ENV_PATH = "/Users/linjia/cursor_ai_space/interview_preparation/ai_ml_interviews/machine_learning/agents_development/crewai_agent/.env"
load_dotenv(ENV_PATH)


def require_key(name: str, prompt: str) -> str:
    val = os.getenv(name)
    if not val:
        val = getpass.getpass(prompt)
    os.environ[name] = val
    return val

require_key("OPENAI_API_KEY", "Enter OPENAI_API_KEY: ")
require_key("SERPER_API_KEY", "Enter SERPER_API_KEY: ")

# Create a search tool
search_tool = SerperDevTool()


Creating the agents and tasks

In [3]:
venue_finder = Agent(
    role="Conference Venue Finder",
    goal="Find the best venue for the upcoming conference",
    backstory=(
        "You are an experienced event planner with a knack for finding the perfect venues. "
        "Your expertise ensures that all conference requirements are met efficiently. "
        "Your goal is to provide the client with the best possible venue options."
    ),
    tools=[search_tool],
    verbose=True
)

In [4]:
venue_quality_assurance_agent = Agent(
    role="Venue Quality Assurance Specialist",
    goal="Ensure the selected venues meet all quality standards and client requirements",
    backstory=(
        "You are meticulous and detail-oriented, ensuring that the venue options provided "
        "are not only suitable but also exceed the client's expectations. "
        "Your job is to review the venue options and provide detailed feedback."
    ),
    tools=[search_tool],
    verbose=True
)

In [5]:
find_venue_task = Task(
    description=(
        "Conduct a thorough search to find the best venue for the upcoming conference in Las Vegas, USA. "
        "Consider factors such as capacity, location, amenities, and pricing. "
        "Use online resources and databases to gather comprehensive information."
    ),
    expected_output=(
        "A list of 5 potential venues with detailed information on capacity, location, amenities, pricing, and availability."
    ),
    tools=[search_tool],
    agent=venue_finder,
)

In [7]:
quality_assurance_review_task = Task(
    description=(
        "Review the venue options provided by the Conference Venue Finder. "
        "Ensure that each venue meets all the specified requirements and standards. "
        "Provide a detailed report on the suitability of each venue."
    ),
    expected_output=(
        "A detailed review of the 5 potential venues, highlighting any issues, strengths, and overall suitability."
    ),
    tools=[search_tool],
    agent=venue_quality_assurance_agent,
)

In [8]:
event_planning_crew = Crew(
  agents=[venue_finder, venue_quality_assurance_agent],
  tasks=[find_venue_task, quality_assurance_review_task],
  verbose=True
)


In [9]:
inputs = {
    "conference_name": "AI Innovations Summit",
    "requirements": "Capacity for 5000, central location, modern amenities, budget up to $50,000"
}

result = event_planning_crew.kickoff(inputs=inputs)


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 3c3bd2cd-faf2-45c4-b519-1a589ac5ee26                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Conference Venue Finder                                                                                 │
│                                                                                                                 │
│  Task: Conduct a thorough search to find the best venue for the upcoming conference in Las Vegas, USA.          │
│  Consider factors such as capacity, location, amenities, and pricing. Use online resources and databases to     │
│  gather comprehensive information.                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Conference Venue Finder                                                                                 │
│                                                                                                                 │
│  Thought: Thought: I need to search for conference venues in Las Vegas, USA, considering key factors such as    │
│  capacity, location, amenities, pricing, and availability. I will start by gathering a list of potential        │
│  venues that meet these requirements.                                                                           │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  {                                                                                                              │
│    "search_query": "top conference venues in Las Vegas capacity amenities pricing availability"                 │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  {'searchParameters': {'q': 'top conference venues in Las Vegas capacity amenities pricing availability',       │
│  'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': 'Top 20 Conference & Summit Venues in  │
│  Las Vegas, NV | PartySlate', 'link':                                                                           │
│  'https://www.partyslate.com/find-venues/conference-summit-venues/near/las-vegas-nv-usa', 'snippet': 'Top Las   │
│  Vegas Conference & Summit Venues · MGM Grand Hotel · Stallion Mountain by Wedgewood Weddings · House of Blues  │
│  Las Vegas · Bel-Aire Backyard & Lounge · Aqua ...', 'position': 1}, {'title': 'Hotels with Meeting Space in    │
│  Las Vegas', 'link': 'https://www.vegasmeansbusiness.com/meeting-facilities/hotels/', 'snippet': 'Ahern Luxury  │
│  Boutique Hotel · Total Sq. Ft.: 34,000 · Sleeping Rooms: 200 · Banquet Capacity: 290 · Theater Capacity: 375   │
│  · Showroom Capacity: 100.', 'position': 2}, {'title': 'Meeting Venues in Las Vegas, NV - 314 Venues | Pricing  │
│  - Eventective', 'link': 'https://www.eventective.com/las-vegas-nv/meeting-venues/?p=4', 'snippet': 'Find and   │
│  contact local Meeting Venues in Las Vegas, NV with pricing and availability for your meeting event. Great for  │
│  meeting planning!', 'position': 3}, {'title': 'Conference Venues for Rent in Las Vegas, NV - Tagvenue',        │
│  'link': 'https://www.tagvenue.com/us/hire/conference-venues/las-vegas', 'snippet': 'Top Conference Venues in   │
│  Las Vegas, NV ; Meeting Room. 65 · Hourly Meeting Room ; Whole Venue. 100 · Lamarre Theater ; Renkus-Heinz     │
│  Theater. 200 · NOTORIETY ; Freeman ...', 'position': 4}, {'title': '7 Best Places To Host Big Conferences In   │
│  Las Vegas - Rozie Synopsis', 'link': 'https://www.roziesynopsis.com/post/host-conferences-las-vegas',          │
│  'snippet': '1) Las Vegas Convention Center · 2) The Venetian Convention & Expo Center · 3) Mandalay Bay        │
│  Convention Center · 4) Sands Expo and Convention Center · 5) ARIA Resort ...', 'position': 5}, {'title': 'Top  │
│  places to host a conference in Las Vegas - ConferenceTap', 'link':                                             │
│  'https://www.conferencetap.com/post/top-places-to-host-a-c...                                                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Venue Quality Assurance Specialist                                                                      │
│                                                                                                                 │
│  Task: Review the venue options provided by the Conference Venue Finder. Ensure that each venue meets all the   │
│  specified requirements and standards. Provide a detailed report on the suitability of each venue.              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Venue Quality Assurance Specialist                                                                      │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  1. Las Vegas Convention Center (LVCC)                                                                          │
│  - Strengths: The LVCC boasts an enormous total meeting and exhibit space of over 2.9 million square feet and   │
│  225 meeting rooms, accommodating a wide range of group sizes from 20 to 3,000 attendees. The largest room      │
│  capacity of approximately 1,440 is sizeable for large plenary sessions. Its AIPC Gold Certification            │
│  underscores its operational and quality standards. The center offers high-tech meeting rooms, ample parking,   │
│  onsite catering, business services, and accessibility via the LVCC Loop transit system which improves          │
│  attendee convenience. Pricing is competitive at $0.35-$0.43 per square foot with gradual increases negotiated  │
│  well into 2035. Its reputation as one of the busiest centers means early booking is essential.                 │
│  - Weaknesses: Due to high demand and busy event calendar (e.g., CES), availability can be a challenge unless   │
│  booked far in advance. The large scale may be overwhelming for smaller or mid-sized events. The largest room   │
│  seating capacity is 1,440, which might be limiting for events requiring even larger plenary spaces.            │
│  - Overall suitability: Excellent choice for very large conferences and trade shows needing vast exhibit space  │
│  and many breakout rooms. Its modern facilities and certifications position it as a top-tier venue. Ideal when  │
│  booking lead time is sufficient.                                                                               │
│                                                                                                                 │
│  2. The Venetian Convention & Expo Center                                                                       │
│  - Strengths: Offers nearly 1.8 million square feet of combined space with over 350 configurable meeting        │
│  rooms, allowing flexible layouts to suit various conference formats from intimate meetings to sizable          │
│  conventions. Being part of The Venetian Resort, it provides luxury suites, concierge services, spa and         │
│  fitness facilities, in-suite dining, extensive dining, and entertainment options which can enhance the         │
│  attendee experience. High-speed internet and full business center capabilities meet modern event technology    │
│  needs.                                                                                                         │
│  - Weaknesses: Pricing tends to be premium due to luxury resort integration, which might stress budgets. Room   │
│  sizes are somewhat smaller (342 to 1,300 square feet) requiring careful planning for larger sessions.          │
│  Popularity and event calendar density mean early bookings are necessary, limiting last-minute availability.    │
│  - Overall suitability: Ideal for clients prioritizing luxury resort amenities alongside conference             │
│  facilities. Best for medium to large events with a strong focus on attendee experience, hospitality, and       │
│  flexibility in event design.                                                                                   │
│                                                                                                                 │
│  3. Mandalay Bay Convention Center                     

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: f9826c90-d051-45a8-ae58-c7aed7e5bb1f                                                                     │
│  Agent: Venue Quality Assurance Specialist                                                                      │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 3c3bd2cd-faf2-45c4-b519-1a589ac5ee26                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: 1. Las Vegas Convention Center (LVCC)                                                            │
│  - Strengths: The LVCC boasts an enormous total meeting and exhibit space of over 2.9 million square feet and   │
│  225 meeting rooms, accommodating a wide range of group sizes from 20 to 3,000 attendees. The largest room      │
│  capacity of approximately 1,440 is sizeable for large plenary sessions. Its AIPC Gold Certification            │
│  underscores its operational and quality standards. The center offers high-tech meeting rooms, ample parking,   │
│  onsite catering, business services, and accessibility via the LVCC Loop transit system which improves          │
│  attendee convenience. Pricing is competitive at $0.35-$0.43 per square foot with gradual increases negotiated  │
│  well into 2035. Its reputation as one of the busiest centers means early booking is essential.                 │
│  - Weaknesses: Due to high demand and busy event calendar (e.g., CES), availability can be a challenge unless   │
│  booked far in advance. The large scale may be overwhelming for smaller or mid-sized events. The largest room   │
│  seating capacity is 1,440, which might be limiting for events requiring even larger plenary spaces.            │
│  - Overall suitability: Excellent choice for very large conferences and trade shows needing vast exhibit space  │
│  and many breakout rooms. Its modern facilities and certifications position it as a top-tier venue. Ideal when  │
│  booking lead time is sufficient.                                                                               │
│                                                                                                                 │
│  2. The Venetian Convention & Expo Center                                                                       │
│  - Strengths: Offers nearly 1.8 million square feet of combined space with over 350 configurable meeting        │
│  rooms, allowing flexible layouts to suit various conference formats from intimate meetings to sizable          │
│  conventions. Being part of The Venetian Resort, it provides luxury suites, concierge services, spa and         │
│  fitness facilities, in-suite dining, extensive dining, and entertainment options which can enhance the         │
│  attendee experience. High-speed internet and full business center capabilities meet modern event technology    │
│  needs.                                                                                                         │
│  - Weaknesses: Pricing tends to be premium due to luxury resort integration, which might stress budgets. Room   │
│  sizes are somewhat smaller (342 to 1,300 square feet) requiring careful planning for larger sessions.          │
│  Popularity and event calendar density mean early bookings are necessary, limiting last-minute availability.    │
│  - Overall suitability: Ideal for clients prioritizing luxury resort amenities alongside conference             │
│  facilities. Best for medium to large events with a strong focus on attendee experience, hospitality, and       │
│  flexibility in event design.                                                                                   │
│                                                       

╭──────────────────────────────────────────────── Tracing Status ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Info: Tracing is disabled.                                                                                     │
│                                                                                                                 │
│  To enable tracing, do any one of these:                                                                        │
│  • Set tracing=True in your Crew/Flow code                                                                      │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file                                                  │
│  • Run: crewai traces enable                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
from IPython.display import Markdown

# Convert the CrewOutput object to a Markdown string
result_markdown = result.raw

# Display the result as Markdown
Markdown(result_markdown)

### Detailed Review of the 5 Potential Venues for the Conference in Las Vegas

#### 1. **Las Vegas Convention Center**
- **Capacity**: Approximately 2.9 million square feet of exhibit space; 225 meeting rooms with seating capacities ranging from 20 to 2,500.
  - **Observation**: The venue offers an extensive amount of space, making it suitable for large-scale conferences. The variety in meeting room sizes allows for flexibility in event planning.
- **Location**: 3150 Paradise Road, Las Vegas, NV 89109
  - **Observation**: Centrally located, making it accessible for attendees traveling from various parts of the city.
- **Amenities**: Preferred catering available, wireless internet, A/V equipment
  - **Observation**: Essential amenities are provided, ensuring a smooth experience for conference attendees.
- **Pricing**: Rates approximately $0.35 per square foot per day
  - **Observation**: Competitive pricing given the extensive space and facilities offered.
- **Availability**: Available for booking through 2023; advance reservations recommended
  - **Observation**: Flexible availability but advance booking is crucial to secure the desired dates.

**Strengths**: Extensive space, variety in meeting room sizes, central location, competitive pricing.
**Issues**: None identified.
**Overall Suitability**: Highly suitable for large conferences with its extensive capacity and comprehensive amenities.

#### 2. **MGM Grand Conference Center**
- **Capacity**: Over 850,000 square feet of flexible meeting space; can accommodate from intimate meetings to large conventions
  - **Observation**: The venue offers a significant amount of flexible space, suitable for various event sizes.
- **Location**: 3799 South Las Vegas Boulevard, Las Vegas, NV 89109
  - **Observation**: Located on the famous Las Vegas Strip, providing easy access and high visibility.
- **Amenities**: Catering services, tech services, gym/fitness center, outdoor pool, shuttle from the airport
  - **Observation**: Comprehensive amenities, including exclusive features like a gym and outdoor pool, enhance attendee experience.
- **Pricing**: Varies based on space and services required
  - **Observation**: Pricing is flexible and can be tailored based on specific event requirements.
- **Availability**: Available for booking; check website for detailed availability
  - **Observation**: Availability is flexible, but it's recommended to check for specific dates.

**Strengths**: Flexible meeting space, prime location, extensive amenities.
**Issues**: Pricing details need to be confirmed based on specific requirements.
**Overall Suitability**: Excellent choice for both small and large events due to its flexibility and range of amenities.

#### 3. **Caesars Palace Conference Center**
- **Capacity**: Total space of 550,000 square feet; largest room is 110,000 square feet; banquet capacity of 9,150, theater capacity of 12,220
  - **Observation**: Large capacity with the ability to host significant events.
- **Location**: 3570 Las Vegas Boulevard South, Las Vegas, NV 89109
  - **Observation**: Central location on the Las Vegas Strip, offering high convenience and visibility.
- **Amenities**: Luxurious guest rooms, concierge services, multiple dining options
  - **Observation**: High-end amenities ensure a luxurious experience for attendees.
- **Pricing**: Varies; promotional pricing may be available for a limited time
  - **Observation**: Potential for promotional pricing can provide cost savings.
- **Availability**: Available for booking; early reservations advised
  - **Observation**: Early booking is recommended to secure dates.

**Strengths**: Large capacity, luxurious amenities, prime location.
**Issues**: Availability may be limited if not booked early.
**Overall Suitability**: Ideal for high-profile events requiring luxurious settings and significant capacity.

#### 4. **The Venetian Expo**
- **Capacity**: Total space of 1,200,000 square feet; largest room is 380,000 square feet; banquet capacity of 1,260, theater capacity of 1,890
  - **Observation**: Offers substantial space, suitable for various types of events.
- **Location**: 201 Sands Avenue, Las Vegas, NV 89169
  - **Observation**: Conveniently located near the Las Vegas Strip.
- **Amenities**: Re-key meeting rooms, unique spaces for general sessions or receptions, valet parking
  - **Observation**: Unique amenities like re-key meeting rooms and valet parking enhance event experience.
- **Pricing**: Varies; check website for detailed rates
  - **Observation**: Pricing needs to be confirmed based on specific requirements.
- **Availability**: Available for booking; check for specific dates
  - **Observation**: Flexible availability, but specific dates should be confirmed.

**Strengths**: Large and flexible space, unique amenities, strategic location.
**Issues**: Pricing details need confirmation.
**Overall Suitability**: Suitable for a wide range of events due to its flexibility and unique amenities.

#### 5. **Bellagio Hotel and Casino Conference Center**
- **Capacity**: Total meeting space of 200,000 square feet; largest room is 45,458 square feet; banquet capacity of 2,970, theater capacity of 4,785
  - **Observation**: Smaller capacity compared to other venues but still substantial for medium-sized events.
- **Location**: 3600 Las Vegas Boulevard South, Las Vegas, NV 89109
  - **Observation**: Located on the Las Vegas Strip, providing high accessibility and visibility.
- **Amenities**: Multiple dining options, elegant rooms, marble bathrooms, wireless internet
  - **Observation**: Luxurious amenities ensure a high-end experience for attendees.
- **Pricing**: Varies based on space and services required
  - **Observation**: Pricing is flexible and can be tailored based on specific event requirements.
- **Availability**: Available for booking; check website for detailed availability
  - **Observation**: Flexible availability but specific dates should be confirmed.

**Strengths**: Luxurious amenities, prime location, flexible pricing.
**Issues**: Smaller capacity compared to other venues.
**Overall Suitability**: Best suited for medium-sized events that prioritize luxury and high-end amenities.

### Final Conclusion:
All five venues offer unique strengths and cater to different needs:
- **Las Vegas Convention Center**: Best for very large conferences needing extensive space.
- **MGM Grand Conference Center**: Offers flexibility for various event sizes with comprehensive amenities.
- **Caesars Palace Conference Center**: Ideal for high-profile, large-capacity events with a luxurious setting.
- **The Venetian Expo**: Versatile and spacious, suitable for a wide range of events.
- **Bellagio Hotel and Casino Conference Center**: Perfect for medium-sized, luxury-focused events.

Contacting the venues directly for specific pricing and availability is recommended to make the final decision based on precise requirements and budget considerations.